Viene richiesto di costruire un sistema di riconoscimento dei volti a mondo chiuso (ovvero i nomi associati ai volti è conosciuto e definito in numero finito) attraverso un sistema di rilevamento dei volti ed uno di classificazione, il tutto applicato ad un dataset generato ad hoc.

L'impementazione è stata eseguita tramite un modello pretrained fornito da Ultralytics chiamato YOLOv8 con un modello che presenta [3,2M di parametri](https://github.com/ultralytics/ultralytics#:~:text=0.99-,3.2,-8.7), anche se è il modello più piccolo mantiene comunque una [ottima precisione](https://github.com/derronqi/yolov8-face#:~:text=92.2-,79.0,-%2D). 
Ai volti rilevati vengono applicate delle modifiche come rotazione, zoom o flip orizzontale per aumentare il numero di dati disponibili alla rete.
La classificazione è affidata ad una rete custom convoluzionale. 


Nella cartella train si trova il dataset diviso in 5 cartelle il cui nome è l'etichetta delle foto al loro interno, è presente il file requirements qualora si volesse eseguire il codice in un nuovo ambiente, il file h5 contentente i pesi del modello (Attenzione, contiene solo i pesi, sarà necessario generare il modello prima).
I file .pkl contengono le matrici serializzate dei volti dei tre set in modo da non dover ricaricare ogni volta i file e cercare i volti (la ricerca su cpu con 4 core impiega circa 200/250ms per volto, che diventano circa 100/120 minuti per le circa 30000 facce del dataset).
Il file .pt invece contiene i pesi per yolov8 con specifica attenzione al rilevamento dei volti 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout,RandomBrightness,RandomRotation,RandomFlip,RandomZoom,Resizing
from keras.utils import to_categorical
from ultralytics import YOLO
import cv2
import numpy as np
import glob
import os
from sklearn.utils import shuffle
from math import floor
from itertools import chain
from tqdm import tqdm
import pickle

YOLO utilizza una rete convoluzionale ed effettua il rilevamento degli oggetti e la loro classificazione "guardando una volta sola" ovvero senza avere la necessità di usare due reti dove la prima propone delle aree e la seconda riconosce gli oggetti.
è una rete estremamente veloce utilizzabile anche in tempo reale, queste caratteristiche l'hanno portata ad essere una rete estremamente utilizzata e nel tempo migliorata, date le 8 versioni in 7 anni.

L'implementazione proposta utilizza la versione sviluppata da Ultralytics con i pesi per i volti.


Viene creato un array con i nomi in modo da poter utilizzare degli interi come label all'interno delle reti

_resizer_ utilizza un layer di keras per fare il ridimensionamento dei volti rilevati mantenendo l'aspect ratio e non applicare quindi warp non voluti

In [ ]:
#https://github.com/derronqi/yolov8-face
#https://docs.ultralytics.com/
detect = YOLO('yolov8n-face.pt')

nomi = ["Davide","Francesco", "Gabriele", "Stefano", "Unknown"]

resizer = Resizing(
    64,
    64,
    interpolation='bilinear',
    crop_to_aspect_ratio=True,
)

Data la struttura descritta in precedenza, vengono caricate le foto e le etichette con un ciclo for che cicla le 5 cartelle e salva le foto ed i relativi tag, questi vengono "incatenati" alle liste precedenti in quanto altrimenti avremmo una struttura annidata, con il metodo chain di itertools invece ci viene restituita una lista in cui sono presenti i vecchi elementi ed i nuovi (ex. chain delle liste [1,2,3,4] e [5,6,7], con il metodo append avremmo una lista di liste [[1,2,3,4],[5,6,7]] mentre con il metodo chain otteniamo [1,2,3,4,5,6,7]).

Vengono quindi cambiati i nomi in indici in quanto le funzioni di keras si aspettano dei valori interi per etichette, questo viene fatto attravero un doppio ciclo for che scorre per ogni tag tutti i nomi e sostituisce qualora trovasse una corrispondenza.
  

In [ ]:
def getDataset(root="train"):

    listDir = os.listdir(root)
    foto = []
    tag = []

    for dir in listDir:
        imgs =  glob.glob(f"{root}/{dir}/*.jpg")
        dirs = [dir]*len(imgs)
        foto = list(chain(foto,imgs))
        tag = list(chain(tag,dirs))
        


    for k in tqdm(range(len(tag)), desc= "Changing names to index"):
        for i in range(len(nomi)):
            if tag[k] == nomi[i]:
                tag[k] = i

    tag = list(map(int, tag))
    foto,tag = shuffle(foto,tag, random_state=42)
    return foto,tag


La funzione _findFaces_ prende in input un frame ed il massimo di volti riconoscibili per quel frame e ritorna la lista di punti per le bbox ed i volti ritagliati, per farlo utilizza il predict della rete YOLO implementata in precedenz

In [ ]:
def findFaces(frame, maxDet = 10):
    img_test = detect.predict(source=frame,max_det=maxDet,verbose=False)
    faces = []
    boxesDetect = []
    for result in img_test:
        boxes = result.boxes  
        boxes = boxes.numpy()
        try:
            face = frame[int(boxes.xyxy[0][1]):int(boxes.xyxy[0][3]),int(boxes.xyxy[0][0]):int(boxes.xyxy[0][2]),:]
            face = resizer(face)
        except:
            face = np.zeros((64,64,3))
        faces = list(chain(faces,face))
        boxesDetect = list(chain(boxesDetect,boxes))
    
    return faces, boxesDetect

Il dataset viene quindi diviso attraverso le percentuali definite dal parametro _percentage_ che di default vuole il 60% del set per il training, il 20% per il validation ed il 20% per il test.

Vengono a questo punto anche sostituiti i path con le relative foto in cui vengono ritagliati i volti presenti (nel caso del training viene limitata questa possibilità ad un rilevamento per foto per evitare problemi qualora dovessero esserci discrepanze tra la lunghezza dei dati e delle label)

In [ ]:
def getSets(x,y, percentage=[0.6,0.2]):
    length = len(x)
    
    trainLen = floor(percentage[0]*length)
    valLen = floor(percentage[1]*length) + trainLen

    for i in tqdm(range(len(x)), desc= "Detecting faces"):
        x[i],_ = findFaces(cv2.imread(x[i]),maxDet=1)

    x = list(map(np.array, x))
    
    x = np.array(x)
    y = np.array(y)
    train = (x[:trainLen],y[:trainLen])
    val = (x[trainLen:valLen],y[trainLen:valLen]) 
    test  = (x[valLen:],y[valLen:])
    return train, val, test

Per rendere la rete più resistente a variabili come inclinazione del volto o variazioni di luce utilizziamo tecniche di data augmentation che prevedono modifica dei dati in ingresso.

I dati dovrebbero essere quindi sostituiti per ottenere un dataset vario, in questa istanza ho deciso di includere sia i dati base che quelli aumentati sia per aumentare i dati di training, sia per non lasciare influenzare tutto l'allenamento da soli parametri randomizzati.

Viene quindi usato un insieme di layer di keras data la facilità di implementazione e la sicura compatibilità con il resto della rete.

In [ ]:
data_augmentation = tf.keras.Sequential([
  RandomFlip("horizontal"),
  RandomRotation(0.2),
  RandomBrightness((-0.2,0.2)),
  RandomZoom(.1, .1)
])

Vengono quindi cercati i dataset e vengono caricati o calcolati qualora sia la prima esecuzione o si voglia effettuare un nuovo training

In [ ]:
if os.path.exists("train.pkl") and os.path.exists("val.pkl") and os.path.exists("test.pkl"):
    with open("train.pkl","rb") as ds:
        train = pickle.load(ds)
    with open("val.pkl","rb") as ds:
        val = pickle.load(ds)
    with open("test.pkl","rb") as ds:
        test = pickle.load(ds)

else: 
    x,y = getDataset()

    train, val, test = getSets(x,y)

    with open("train.pkl","wb") as ds:
        (X_train, Y_train) = train
        Xtrain = []
        Ytrain = []
        for i in tqdm(range(len(X_train)), desc= "data augmentation"):
                Xtrain.append(data_augmentation(X_train[i]))
                Xtrain.append(X_train[i])
                Ytrain.append(Y_train[i])
                Ytrain.append(Y_train[i])

        X_train = np.array(Xtrain)
        Y_train = np.array(Ytrain)
        train = (X_train,Y_train)
        pickle.dump(train,ds)

    with open("val.pkl","wb") as ds:
        pickle.dump(val,ds)

    with open("test.pkl","wb") as ds:
        pickle.dump(test,ds)

Le tuple vengono quindi divise tra dati e label

In [ ]:
(X_train, Y_train) = train
(X_val,Y_val) = val
(X_test, Y_test) = test

Il classificatore è una rete che, attraverso diversi layer, arriva a determinare la classe di un oggetto che gli viene sottoposto, nell'implementazione proposta questo viene svolto da una rete convoluzionale di profondità cinque con diverso numero di filtri e dimensione di finestra intervallati da maxpooling e dropout.

In [ ]:

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=[keras.metrics.CategoricalAccuracy()]
)

model.summary()

Se non si trovasse il file partirebbe il fit della rete con il relativo callback

In [ ]:
if os.path.exists("pesiClassificatore.h5"):
    model.load_weights('pesiClassificatore.h5')

else:
    callback = keras.callbacks.EarlyStopping(monitor= "val_loss", patience=3,restore_best_weights=True)
    model.fit(
        X_train, to_categorical(Y_train), epochs=100, 
        batch_size=64, shuffle=True, 
        validation_data=(X_val,to_categorical(Y_val)),
        callbacks=callback
        )

    model.save_weights('pesiClassificatore.h5')


Il metodo evaluate ci permette di verificare come è andato l'allenamento utilizzando l'ultima porzione di dati per verificare quanto la rete prevede correttamente su dati mai visti

In [ ]:
results = model.evaluate(
  X_test,
  to_categorical(Y_test)
)
print(results)

Funzione per classificare i frame, come lo faccio, cosa faccio e cosa ritorno

Per classificare i volti presenti in ogni frame viene invocata la funzione _classificatore_ che utilizza nuovamente il metodo predict di YOLO per ottenere una lista dei singoli volti da passare dopo al classificatore definito precedentemente. 

Una volta raccolte le bbox e le label corrispondenti vengono modificati i frame originali disegnando le bbox e scrivendo le label.

Viene quindi ritornato l'array di frame per essere salvato 


Nota: la documentazione suggerisce di usare il modello come funzione (model(dato)) nel caso in cui sia solo uno il valore da predirre, mentre usare il metodo predict nel caso in cui ci sia una batch di dati da predirre 

In [ ]:
def classificatore(frames):
    font = cv2.FONT_HERSHEY_SIMPLEX
    for f in tqdm(range(len(frames)), desc="Face recognition per frame"):
        faces,boxes = findFaces(frames[f])
        predict=model(faces)
        for (boxe,pred) in zip(boxes, predict):
            frames[f] = cv2.putText(frames[f], nomi[np.argmax(pred)] , (int(boxe.xyxy[0][0])-5,int(boxe.xyxy[0][1])-5),font, 1,(255,255,255),2)
            frames[f] = cv2.rectangle(frames[f], (int(boxe.xyxy[0][0]), int(boxe.xyxy[0][1])), (int(boxe.xyxy[0][2]), int(boxe.xyxy[0][3])), (255, 0, 255), 4)
    return frames
    

Viene caricato il video e smembrato in frames con 3 canali (BGR)

In [ ]:
video = cv2.VideoCapture("Video finale senza riconoscimento.mp4")
frames = []
if (video.isOpened() == False):
    print("Error opening video file")
while(video.isOpened()):
  ret, frame = video.read()
  if ret == True:
        frames.append(frame)
  else:
      break


Raccogliamo quindi i frame classificati (Vengono tagliati ai primi 1500 frames per limiti di upload di github)

In [ ]:
results = classificatore(frames)

A questo punto vengono raccolti altezza e larghezza del singolo frame e si prepara dinamicamente il salvataggio dei frame modificati in video

In [ ]:
height, width, channels = results[0].shape
size = (width,height)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out15 = cv2.VideoWriter('project_video_finale.mp4',fourcc, 15, size)

for i in tqdm(range(len(results)), desc="Saving frames into video"):
    out15.write(results[i])
out15.release()

Nel caso si volesse provare in tempo reale

In [ ]:

def classificatoreIRT(frame):
    font = cv2.FONT_HERSHEY_SIMPLEX

    faces,boxes = findFaces(frame)    
    predict=model(faces)
    try:
        for (boxe,pred) in zip(boxes, predict):
            frame = cv2.putText(frame, nomi[np.argmax(pred)] , (int(boxe.xyxy[0][0])-5,int(boxe.xyxy[0][1])-5),font, 1,(255,255,255),2)
            frame = cv2.rectangle(frame, (int(boxe.xyxy[0][0]), int(boxe.xyxy[0][1])), (int(boxe.xyxy[0][2]), int(boxe.xyxy[0][3])), (255, 0, 255), 4)
    except:
        pass    
    return frame


camera = cv2.VideoCapture(0)
if not camera.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = camera.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    frame = classificatoreIRT(frame)
    cv2.imshow('Capture - Face detection', frame)
    if cv2.waitKey(10) == 27:
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break

Sono state testate diverse reti convoluzionali la cui topografia e risultati sono riportati di seguito:
<details><summary>Reti*</summary>
    <details><summary>Rete0</summary>
    Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3))<br>   
    Conv2D(64, (5, 5), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Dropout(0.1)<br>
    Conv2D(64, (3, 3), activation='relu')<br>   
    Conv2D(64, (3, 3), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Conv2D(64, (3, 3), activation='relu')<br>   
    Dropout(0.1)<br>
    Flatten()<br>
    Dense(5, activation='softmax')<br> 
    </details>
    <details><summary>Rete1</summary>
    Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3))<br>
    Conv2D(64, (5, 5), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Flatten()<br>
    Dense(5, activation='softmax')<br>
    </details>
    <details><summary>Rete2</summary>
    Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3))<br>
    MaxPool2D(pool_size=(2,2))<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Flatten()<br>
    Dense(5, activation='softmax')<br>
    </details>
    <details>
    <summary>Rete3</summary>
    Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3))<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Flatten()<br>
    Dense(5, activation='softmax')<br>
    </details>
    <details>
    <summary>Rete4</summary>
    Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3))<br>
    Conv2D(64, (5, 5), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Dropout(0.1)<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    MaxPool2D(pool_size=(2,2))<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Dropout(0.1)<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Conv2D(64, (3, 3), activation='relu')<br>
    Flatten()<br>
    Dense(5, activation='softmax')<br>
    </details>
</details> 


Le reti proposte hanno dato i seguenti risultati

| #Rete | Loss** | Metrics** | # Layer Convoluzionali | # Layer Dropout | # Layer Maxpool |
| --- | --- | --- | --- | --- | --- |
| 0   | 0.1490 | 0.9452 | 5 | 2 | 2 |
| 1   | 0.4058 | 0.8623 | 5 | 0 | 2 |
| 2   | 0.0914 | 0.9647 | 3 | 0 | 2 |
| 3   | 0.2000 | 0.9349 | 3 | 0 | 0 |
| 4   | 0.0910 | 0.9622 | 7 | 2 | 2 |

Il modello di loss utilizzato è il "categorical_crossentropy", mentre la metrica è fornita dal modulo metrics di keras ed è la CategoricalAccuracy

Come è possibile notare dai valori della loss e della metrica, abbiamo risultati molto simili tra la rete 2 e la rete 4, pertanto, dato il minore numero di strati della rete 2 questa verrà preferita tra tutte



*Le reti sono cosi riprodotte per facilitarne l'uso in futuro in quanto basterà copiare le istruzioni direttamente
**Valore troncato al quarto decimale

La versione più recente di questo progetto è disponibile [qui](https://github.com/Montenigri/VisioneArtificiale/tree/main/Riconoscimento%20volti%20deep)

Fonti:

[Benchmark](https://github.com/derronqi/yolov8-face)

[Numero parametri](https://github.com/ultralytics/ultralytics)

[YOLO](https://12ft.io/proxy?&q=https%3A%2F%2Ftowardsdatascience.com%2Fyolo-you-only-look-once-real-time-object-detection-explained-492dc9230006)

[Predict vs Funzione](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict)